In [2]:
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder , PolynomialFeatures 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
train = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/train.csv')
test = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/test.csv')
sample_submission = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/sample_submission.csv')

In [3]:
data = pd.concat([train , test] , axis= 0)
test_id = test["Id"]
data.drop(["Id"] ,axis=1, inplace=True)
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [4]:
str_cols = data.select_dtypes("O").columns
num_cols = list(set(data.columns) - set(str_cols))
no_cols = ["MiscFeature" , "Fence" , "GarageCond" , "GarageQual" , "GarageFinish" , "FireplaceQu" , "BsmtFinType2" , "BsmtExposure", "BsmtCond" , "BsmtQual" , "Alley"]
for col in test.columns:
    if col != "SalePrice":
        if col in no_cols:
            data[col]=data[col].fillna("no")
        elif col in str_cols:
            data[col]=data[col].fillna(data[col].mode()[0])
        elif col in num_cols:
            data[col]=data[col].fillna(data[col].median())

In [5]:
ohe  = OneHotEncoder(drop = "first")
ohe.fit(data[str_cols])
ohe_data=ohe.transform(data[str_cols]).toarray()
ohe_df = pd.DataFrame(ohe_data , columns = ohe.get_feature_names_out())

In [6]:
if list(str_cols) in list(data.columns):
    data.drop(str_cols , inplace=True ,axis = 1)
data.index=range(len(data))

ohe_df.index=range(len(ohe_df))

In [7]:
data=pd.concat([data, ohe_df], axis=1)

In [8]:
from sklearn.compose import ColumnTransformer

train=data[data['SalePrice'].notnull()]
test=data[data['SalePrice'].isnull()]
test.drop(['SalePrice'] , inplace=True , axis = 1)

X=train.drop('SalePrice', axis=1)
Y=train['SalePrice']

# One-hot encode categorical variables
ct = ColumnTransformer([('encoder', OneHotEncoder(), X.select_dtypes(include=['object']).columns)], remainder='passthrough')
X = ct.fit_transform(X)



In [9]:
def Poly_F(x , y , i ):
    pf = PolynomialFeatures(degree=i)
    pf.fit(x)
    x_poly = pd.DataFrame(pf.transform(x) , columns = pf.get_feature_names_out())
    X_train , X_test , Y_train , Y_test = train_test_split(x_poly , y , test_size=0.3 , random_state=42)
    lr = LinearRegression()
    lr.fit(X_train , Y_train)
    rmse_t = np.sqrt(mean_squared_error(Y_train , lr.predict(X_train)))
    rmse= np.sqrt(mean_squared_error(Y_test , lr.predict(X_test)))
    return rmse_t , rmse

In [10]:
result = {'degree':[] ,'train':[] , 'test':[] }

In [ ]:
for i in range(1,10):
    rmset , rmse = Poly_F(X , Y , i)
    result['degree'].append(i)
    result['train'].append(rmset)
    result['test'].append(rmse)



: 

In [ ]:


filename = "/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/sample_submission2.csv"
import os

test_ohe = ct.transform(test)

test_pred = lr.predict(test_ohe)

sample_submission['SalePrice'] = pd.Series(test_pred)

if os.path.exists(filename):
    os.remove(filename)
sample_submission.to_csv(filename, index=False) 
sample_submission.head()

,Id,SalePrice
0,1461,112861.707900
1,1462,159090.872956
2,1463,186798.287295
3,1464,197192.639890
4,1465,205107.352381
